<a href="https://colab.research.google.com/github/macontreras04/cpf/blob/main/HW_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 63.3 MB/s eta 0:00:00


In [3]:
from QuantLib import *

In [11]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from collections import namedtuple
import math

In [5]:
today = Date(18, April, 2023);
settlement= Date(19, April, 2024);
Settings.instance().evaluationDate = today;
term_structure = YieldTermStructureHandle(
FlatForward(settlement,0.04875825,Actual365Fixed())
)
index = Euribor1Y(term_structure)

In [6]:
CalibrationData = namedtuple("CalibrationData",
"start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
CalibrationData(2, 4, 0.1108),
CalibrationData(3, 3, 0.1070),
CalibrationData(4, 2, 0.1021),
CalibrationData(5, 1, 0.1000 )]

In [7]:
def create_swaption_helpers(data, index, term_structure, engine):
  swaptions = []
  fixed_leg_tenor = Period(1, Years)
  fixed_leg_daycounter = Actual360()
  floating_leg_daycounter = Actual360()
  for d in data:
    vol_handle = QuoteHandle(SimpleQuote(d.volatility))
    helper = SwaptionHelper(Period(d.start, Years),
    Period(d.length, Years),
    vol_handle,
    index,
    fixed_leg_tenor,
    fixed_leg_daycounter,
    floating_leg_daycounter,
    term_structure
    )
    helper.setPricingEngine(engine)
    swaptions.append(helper)
  return swaptions

In [8]:
def calibration_report(swaptions, data):
  columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Er\
  ror Price", "Rel Error Vols"]
  report_data = []
  cum_err = 0.0
  cum_err2 = 0.0
  for i, s in enumerate(swaptions):
    model_price = s.modelValue()
    market_vol = data[i].volatility
    black_price = s.blackPrice(market_vol)
    rel_error = model_price/black_price - 1.0
    implied_vol = s.impliedVolatility(model_price,
    1e-5, 50, 0.0, 0.50)
    rel_error2 = implied_vol/market_vol-1.0
    cum_err += rel_error*rel_error
    cum_err2 += rel_error2*rel_error2
    report_data.append((model_price, black_price, implied_vol,
    market_vol, rel_error, rel_error2))
  print ("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
  print( "Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
  return DataFrame(report_data,columns= columns, index=['']*len(report_data))

In [9]:
model = HullWhite(term_structure);
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print ("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.05247, sigma = 0.00688


In [12]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.29531
Cumulative Error Vols : 0.29616


,Model Price,Market Price,Implied Vol,Market Vol,Rel Er ror Price,Rel Error Vols
,0.010269,0.013374,0.088110,0.1148,-0.232147,-0.232495
,0.011277,0.012314,0.101447,0.1108,-0.084190,-0.084417
,0.010099,0.010047,0.107550,0.1070,0.005121,0.005141
,0.007585,0.006969,0.111170,0.1021,0.088392,0.088834
,0.004128,0.003635,0.113644,0.1000,0.135600,0.136438


# **HW from scratch**

In [13]:
import numpy as np
from scipy.optimize import minimize
import math

# Flat yield curve function
def flat_yield_curve(rate, t):
    return np.exp(-rate * t)

# Example usage: flat yield curve with 4.875825% rate
rate = 0.04875825
t = 1.0  # 1 year maturity
print(flat_yield_curve(rate, t))  # Outputs the discount factor


0.9524113473130424


In [14]:
def hull_white_zero_coupon_bond(a, sigma, rate, t, T):
    # Implement the Hull-White model's zero-coupon bond pricing formula
    B = (1 - np.exp(-a * (T - t))) / a
    A = np.exp((B - (T - t)) * (rate - sigma**2 / (2 * a**2)) - (sigma**2 * B**2) / (4 * a))
    return A * flat_yield_curve(rate, T - t)

# Example usage
a = 0.05
sigma = 0.01
t = 0  # Current time
T = 1  # Maturity in 1 year
print(hull_white_zero_coupon_bond(a, sigma, rate, t, T))

0.9512854671198083


In [15]:
def black_model_swaption_price(forward_rate, strike, volatility, maturity, notional):
    # Black model formula for swaption pricing
    d1 = (np.log(forward_rate / strike) + 0.5 * volatility**2 * maturity) / (volatility * np.sqrt(maturity))
    d2 = d1 - volatility * np.sqrt(maturity)
    nd1 = 0.5 * (1 + math.erf(d1 / np.sqrt(2)))
    nd2 = 0.5 * (1 + math.erf(d2 / np.sqrt(2)))
    return notional * (forward_rate * nd1 - strike * nd2)

# Example parameters
forward_rate = 0.05  # Forward rate for the swap
strike = 0.045  # Strike rate of the swaption
volatility = 0.01  # Implied volatility
maturity = 1.0  # Maturity in years
notional = 1000000  # Notional value
print(black_model_swaption_price(forward_rate, strike, volatility, maturity, notional))


5000.000000000005


In [19]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import math

# Flat yield curve function
def flat_yield_curve(rate, t):
    return np.exp(-rate * t)

# Hull-White zero-coupon bond pricing function using the Hull-White model formula
def hull_white_zero_coupon_bond(a, sigma, rate, t, T):
    B = (1 - np.exp(-a * (T - t))) / a
    A = np.exp((B - (T - t)) * (rate - (sigma**2 / (2 * a**2))) - (sigma**2 * B**2) / (4 * a))
    return A * flat_yield_curve(rate, T - t)

# Function to calculate the forward swap rate given Hull-White model
def forward_swap_rate(a, sigma, rate, t, T, S):
    P0T = hull_white_zero_coupon_bond(a, sigma, rate, t, T)
    P0S = hull_white_zero_coupon_bond(a, sigma, rate, t, S)
    return (P0T / P0S - 1) / (S - T)

# Black model for swaption pricing
def black_model_swaption_price(forward_rate, strike, volatility, maturity, notional):
    d1 = (np.log(forward_rate / strike) + 0.5 * volatility**2 * maturity) / (volatility * np.sqrt(maturity))
    d2 = d1 - volatility * np.sqrt(maturity)
    nd1 = norm.cdf(d1)
    nd2 = norm.cdf(d2)
    return notional * (forward_rate * nd1 - strike * nd2)

# Objective function for calibration
def calibration_objective(params, market_vols, swaption_data, rate):
    a, sigma = params
    model_prices = []
    for start, length, market_vol in swaption_data:
        T = start
        S = start + length
        strike = market_vol  # Using market vol as a proxy for the strike (fixed for now)
        notional = 1.0  # Simplified notional value

        # Calculate the forward rate using the Hull-White model
        forward_rate = forward_swap_rate(a, sigma, rate, 0, T, S)

        # Calculate model price using the Black model for swaptions
        model_price = black_model_swaption_price(forward_rate, strike, sigma, T, notional)
        model_prices.append(model_price)

    # Calculate the sum of squared errors between model and market prices
    error = np.sum((np.array(model_prices) - np.array(market_vols))**2)
    return error

# Sample market data for calibration
swaption_data = [
    (1, 5, 0.1148),
    (2, 4, 0.1108),
    (3, 3, 0.1070),
    (4, 2, 0.1021),
    (5, 1, 0.1000)
]
market_vols = [d[2] for d in swaption_data]

# Initial guess for optimization
initial_params = [0.03, 0.01]  # Initial guesses for a and sigma
rate = 0.04875825

# Bounds to ensure the parameters remain positive
bounds = [(0.0001, None), (0.0001, None)]

# Perform calibration
result = minimize(calibration_objective, initial_params, args=(market_vols, swaption_data, rate), method='L-BFGS-B', bounds=bounds)
a_calibrated, sigma_calibrated = result.x

# Display calibrated parameters
a_calibrated, sigma_calibrated



(0.03, 0.01)